In [88]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2

In [90]:
import pandas as pd
import numpy as np

from src.features import build_features

ANIO_INICIO = 2017
ANIO_TERMINO = 2035

COLUMNAS_POBLACION = [f"Poblacion {i}" for i in range(ANIO_INICIO, ANIO_TERMINO + 1)]

COMUNAS_SSMO = [
    "Peñalolén",
    "Macul",
    "Ñuñoa",
    "La Reina",
    "Providencia",
    "Las Condes",
    "Vitacura",
    "Lo Barnechea",
    "Isla de Pascua",
]

In [91]:
ruta_base_de_datos = (
    "../data/raw/3.0-planilla_poblacion_total/estimaciones-y-proyecciones-2002-2035-comunas.xlsx"
)

# Lee la base de datos
df = pd.read_excel(ruta_base_de_datos).iloc[:-2]

# Renombra columna de hombres o mujeres
df = df.rename(columns={"Sexo\n1=Hombre\n2=Mujer": "hombre_mujer"})

# Indica si es adulto o infantil
df["grupo_etario_poblacion"] = np.where(df["Edad"] >= 15, "Adulto", "Infantil")

# Cálculo de Personas por Estrato

Aquí se quiere calcular la cantidad de personas que hay en los siguientes estratos:

1. Hombres y Mujeres
2. Hombres
3. Mujeres
4. Población sobre 45 años
5. Población sobre 60 años
6. Población entre 60 a 70 años
7. Población recién nacidos
8. Población sobre 65 años

Para cada estrato se calculará la población que hay para el SSMO, Región Metropolitana y el País.

In [92]:
ESTRATOS_A_CALCULAR = {}

# Obtiene estratos a calcular
df_pais = df.copy()
ESTRATOS_A_CALCULAR["Pais"] = df_pais

for region in df_pais["M"].unique():
    df_region = df_pais.query("M == @region").copy()
    ESTRATOS_A_CALCULAR[region] = df_region

df_ssmo = df_pais[df_pais["Nombre Comuna"].isin(COMUNAS_SSMO)]
ESTRATOS_A_CALCULAR["SSMO"] = df_ssmo

## Definición de grupos etarios a calcular

En el próximo diccionario se definirán todos los grupos etarios que se quieren calcular para cada estrato previamente definidos.

In [93]:
# Define todos los grupos etarios a calcular.
query_strings = {
    "pais": "", # Todo el pais
    "hombres": "hombre_mujer == 1", # Hombres
    "mujeres": "hombre_mujer == 2", # Mujeres
    "recien_nacidos_vivos": "Edad == 0", # Recien Nacidos Vivos
    "sobre_15": "Edad > 15", # Sobre 15 anios
    "sobre_20": "Edad > 20", # Sobre 20 anios
    "sobre_45": "Edad > 45", # Sobre 45 anios
    "sobre_60": "Edad > 60", # Sobre 60 anios
    "sobre_65": "Edad > 65", # Sobre 65 anios
    "entre_15_a_24_anios": "Edad >= 15 and Edad <= 24", # Entre 15 y 24 anios
    "entre_25_a_44_anios": "Edad >= 25 and Edad <= 44", # Entre 25 y 44 anios
    "entre_45_a_59_anios": "Edad >= 45 and Edad <= 59", # Entre 45 y 59 anios
    "entre_60_y_70": "Edad >= 60 and Edad <= 70", # Entre 60 y 70 anios
    "entre_60_a_74_anios": "Edad >= 60 and Edad <= 74", # Entre 60 a 74 anios
    "mayores_a_75_anios": "Edad >= 75", # Mayores a 75 anios

}

In [94]:
results = build_features.iterate_queries(ESTRATOS_A_CALCULAR, query_strings, COLUMNAS_POBLACION)